In [1]:
import tensorflow.contrib.eager as tfe
import tensorflow as tf
import numpy as np
import utility
mnist_folder = 'data/mnist'
tf.enable_eager_execution()
mnist_folder = 'data/mnist'
layer1_size = 300
layer2_size = 100
input_size = 28 * 28
output_size = 10  # 1 hot encoder


class LeNet300(tf.keras.Model):

    def __init__(self):
        super(LeNet300, self).__init__()
        self.layer1 = tf.layers.Dense(layer1_size, activation=tf.nn.relu)
        self.layer2 = tf.layers.Dense(layer2_size, activation=tf.nn.relu)
        self.out = tf.layers.Dense(output_size)

    def call(self, x, **kwargs):
        return self.out(self.layer2(self.layer1(x)))
# Define the loss function
def loss(net, x, y):
    ypred = net(x)
    l = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=ypred))
    return l

def print_zero_stat(net):
    [l1,l1_b] = net.layer1.get_weights()
    [l2 , l2_b]= net.layer2.get_weights()
    [out, out_b] = net.out.get_weights()
    print('Layer1 zeros:',np.count_nonzero(l1==0),' on ',l1.shape[0]*l1.shape[1])
    print('Layer1 bias zeros :',np.count_nonzero(l1_b == 0),' on ',l1_b.shape[0])
    print('Layer2 zeros :',np.count_nonzero(l2 == 0),' on ',l2.shape[0]*l2.shape[1])
    print('Layer2 bias zeros :',np.count_nonzero(l2_b == 0),' on ',l2.shape[0])
    print('Out zeros :',np.count_nonzero(out == 0),' on ',out.shape[0]*out.shape[1])
    print('Out bias zeros :',np.count_nonzero(out_b == 0),' on ',out_b.shape[0])



In [2]:
# get the pruned NN
[(xtrain, ytrain), (xval, yval), (xtest, ytest)] = utility.read_mnist(mnist_folder, flatten=True,num_train=-1)
net = LeNet300()
loss_grad = tfe.implicit_value_and_gradients(loss)
opt = tf.train.AdamOptimizer(0.001)
root = tf.train.Checkpoint(optimizer=opt,
                           model=net,
                           optimizer_step=tf.train.get_or_create_global_step())
root.restore(tf.train.latest_checkpoint('checkpoint-lenet300'))

train_data = tf.data.Dataset.from_tensor_slices((xtrain, ytrain)).shuffle(1000).batch(32, drop_remainder=True)
ypred = tf.nn.softmax(net(tf.constant(xtest)))
ypred = tf.argmax(ypred, axis=1)

ytest = tf.argmax(ytest, axis=1)
tmp = tf.cast(tf.equal(ypred, ytest), tf.float32)
print('before quantization \naccuracy: ', tf.reduce_mean(tmp).numpy())
print_zero_stat(net)

            

before quantization 
accuracy:  0.9721
Layer1 zeros: 230068  on  235200
Layer1 bias zeros : 300  on  300
Layer2 zeros : 28566  on  30000
Layer2 bias zeros : 97  on  300
Out zeros : 777  on  1000
Out bias zeros : 10  on  10


In [3]:
from sklearn.cluster import KMeans



In [4]:
def apply_cluster_to_gradient(gradient,cluster,label):
    new_gradient = np.copy(gradient)
    sx=new_gradient.shape[0]
    sy=new_gradient.shape[1]
    grad_dic={}
    for i in range(sx):
        for j in range(sy):
            if (label[i*j] not in grad_dic):
                grad_dic[label[i*j]]=gradient[i,j]
            else:
                grad_dic[label[i*j]]+=gradient[i,j]
    for i in range(sx):
        for j in range(sy):
            new_gradient[i,j] = grad_dic[label[i*j]]
                
    return new_gradient
    
    

In [ ]:
for Xtrain_t, ytrain_t in tfe.Iterator(train_data):
    
    #w1 = net.layer1.get_weights()[0]
    #w2 = net.layer1.get_weights()[1]
    
    #w_mod=w1.reshape(-1,1)
    #print(w_mod.shape)
    #kmeans = KMeans(n_clusters=50, random_state=0)
    #kmeans.fit(w_mod)
    #cc=kmeans.cluster_centers_
    #lab = kmeans.labels_
    #w1_cls = np.copy(w1)
    #tf.Variable(w1_cls)
    
    sx=w1.shape[0]
    sy=w1.shape[1]
    for i in range(sx):
        for j in range(sy):
            w1_cls[i,j] = kmeans.cluster_centers_[lab[i*j]]
    #current_loss, grads = loss_grad(net, Xtrain_t, ytrain_t)
    #print(grads[0])
    print('a')
    #new_grad=apply_cluster_to_gradient(grads[0][0],cc,lab)
    print('b')
    net.set_weights([w1,w2])
    #grads[0]=(new_grad,tf.contrib.eager.Variable(w1_cls))
    
    #opt.apply_gradients(grads)
    break
#opt.apply_gradients(grads)

In [11]:
w1 = net.layer1.get_weights()[0]
w2 = net.layer1.get_weights()[1]

w3 = net.layer2.get_weights()[0]
w4 = net.layer2.get_weights()[1]

w5 = net.out.get_weights()[0]
w6 = net.out.get_weights()[1]
bits=5


min_ = w1.min()
max_ = w1.max()
space = np.linspace(min_, max_, num=2**bits)
kmeans = KMeans(n_clusters=len(space), init=space.reshape(-1,1), n_init=1, precompute_distances=True, algorithm="full")
kmeans.fit(w1.reshape(-1,1))
new_weight1 = kmeans.cluster_centers_[kmeans.labels_].reshape(w1.shape)
print(new_weight1.shape)


min_ = w3.min()
max_ = w3.max()
space = np.linspace(min_, max_, num=2**bits)
kmeans = KMeans(n_clusters=len(space), init=space.reshape(-1,1), n_init=1, precompute_distances=True, algorithm="full")
kmeans.fit(w3.reshape(-1,1))
new_weight3 = kmeans.cluster_centers_[kmeans.labels_].reshape(w3.shape)
print(new_weight3.shape)


min_ = w5.min()
max_ = w5.max()
space = np.linspace(min_, max_, num=2**bits)
kmeans = KMeans(n_clusters=len(space), init=space.reshape(-1,1), n_init=1, precompute_distances=True, algorithm="full")
kmeans.fit(w5.reshape(-1,1))
new_weight5 = kmeans.cluster_centers_[kmeans.labels_].reshape(w5.shape)
print(new_weight5.shape)

(784, 300)
(300, 100)
(100, 10)


In [12]:
net.layer1.set_weights([new_weight1,w2])
net.layer2.set_weights([new_weight3,w4])
net.out.set_weights([new_weight5,w6])

In [13]:
ypred = tf.nn.softmax(net(tf.constant(xtest)))
ypred = tf.argmax(ypred, axis=1)
tmp = tf.cast(tf.equal(ypred, ytest), tf.float32)
print('after quantization \naccuracy: ', tf.reduce_mean(tmp).numpy())
print_zero_stat(net)

after quantization 
accuracy:  0.9679
Layer1 zeros: 0  on  235200
Layer1 bias zeros : 0  on  300
Layer2 zeros : 0  on  30000
Layer2 bias zeros : 6  on  300
Out zeros : 0  on  1000
Out bias zeros : 0  on  10


In [14]:
print(net.get_weights()[0])

[[-0.00109041 -0.00109041 -0.00109041 ... -0.00109041 -0.00109041
  -0.00109041]
 [-0.00109041 -0.00109041 -0.00109041 ... -0.00109041 -0.00109041
  -0.00109041]
 [-0.00109041 -0.00109041 -0.00109041 ... -0.00109041 -0.00109041
  -0.00109041]
 ...
 [-0.00109041 -0.00109041 -0.00109041 ... -0.00109041 -0.00109041
  -0.00109041]
 [-0.00109041 -0.00109041 -0.00109041 ... -0.00109041 -0.00109041
  -0.00109041]
 [-0.00109041 -0.00109041 -0.00109041 ... -0.00109041 -0.00109041
  -0.00109041]]


In [15]:
kmeans = KMeans(n_clusters=50, random_state=0)
kmeans.fit(w1.reshape((-1,1)))
new_weight = kmeans.cluster_centers_[kmeans.labels_].reshape(784,300)
net.layer1.set_weights([new_weight,w2])

/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (32) found smaller than n_clusters (50). Possibly due to duplicate points in X.
  return_n_iter=True)


In [16]:
ypred = tf.nn.softmax(net(tf.constant(xtest)))
ypred = tf.argmax(ypred, axis=1)
tmp = tf.cast(tf.equal(ypred, ytest), tf.float32)
print('after quantization \naccuracy: ', tf.reduce_mean(tmp).numpy())
print_zero_stat(net)

after quantization 
accuracy:  0.9679
Layer1 zeros: 0  on  235200
Layer1 bias zeros : 0  on  300
Layer2 zeros : 0  on  30000
Layer2 bias zeros : 6  on  300
Out zeros : 0  on  1000
Out bias zeros : 0  on  10


In [17]:
print(net.get_weights()[0])

[[-0.00107697 -0.00107697 -0.00107697 ... -0.00107697 -0.00107697
  -0.00107697]
 [-0.00107697 -0.00107697 -0.00107697 ... -0.00107697 -0.00107697
  -0.00107697]
 [-0.00107697 -0.00107697 -0.00107697 ... -0.00107697 -0.00107697
  -0.00107697]
 ...
 [-0.00107697 -0.00107697 -0.00107697 ... -0.00107697 -0.00107697
  -0.00107697]
 [-0.00107697 -0.00107697 -0.00107697 ... -0.00107697 -0.00107697
  -0.00107697]
 [-0.00107697 -0.00107697 -0.00107697 ... -0.00107697 -0.00107697
  -0.00107697]]
